In [10]:
import os
import shutil
import time
from pathlib import Path
from dotenv import load_dotenv
from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials
from PIL import Image
import requests

load_dotenv()

FACE_API_KEY = os.getenv('API_KEY_FACE')
FACE_ENDPOINT = os.getenv('URL_FACE')
VISION_API_KEY = os.getenv('API_KEY')  # assuming same key for Computer Vision
VISION_ENDPOINT = os.getenv('URL')

train_folder = 'People_Train'     # training images subfolders per person
source_folder = 'People_Input'    # photos to organize
destination_folder = 'People_Output'  # output folder for organized photos

PERSON_GROUP_ID = 'myfacegrouptest'  # your Person Group ID (lowercase alphanumeric)

face_client = FaceClient(FACE_ENDPOINT, CognitiveServicesCredentials(FACE_API_KEY))



In [11]:
def train_person_group():
    print(f"Deleting if exists then creating person group: {PERSON_GROUP_ID}")
    try:
        face_client.person_group.delete(PERSON_GROUP_ID)
        print("Deleted existing person group.")
    except Exception:
        print("Person group did not exist or could not be deleted. Proceeding to create.")

    face_client.person_group.create(person_group_id=PERSON_GROUP_ID, name=PERSON_GROUP_ID)

    for person_name in os.listdir(train_folder):
        person_path = os.path.join(train_folder, person_name)
        if not os.path.isdir(person_path):
            continue

        print(f"Adding person: {person_name}")
        person = face_client.person_group_person.create(PERSON_GROUP_ID, person_name)

        for image_name in os.listdir(person_path):
            if not image_name.lower().endswith(('jpg', 'jpeg', 'png', 'bmp')):
                continue
            image_path = os.path.join(person_path, image_name)
            with open(image_path, 'rb') as img_stream:
                face_client.person_group_person.add_face_from_stream(
                    PERSON_GROUP_ID, person.person_id, img_stream
                )
                print(f" - Added face image: {image_name}")

    print("Training Person Group...")
    face_client.person_group.train(PERSON_GROUP_ID)

    while True:
        status = face_client.person_group.get_training_status(PERSON_GROUP_ID)
        print(f"Training status: {status.status}")
        if status.status.lower() == 'succeeded':
            print("Training complete!")
            break
        elif status.status.lower() == 'failed':
            raise Exception("Person Group training failed")
        time.sleep(5)


# Run training once before your organizing process
train_person_group()


Deleting if exists then creating person group: myfacegrouptest
Person group did not exist or could not be deleted. Proceeding to create.


APIErrorException: (InvalidRequest) Invalid request has been sent.

In [ ]:
def approximately_same_face(rect1, rect2, tolerance=20):
    for key in ['left', 'top', 'width', 'height']:
        if abs(rect1.get(key, 0) - rect2.get(key, 0)) > tolerance:
            return False
    return True

def get_celebrities(image_path):
    """
    Detects celebrity faces in a local image using Azure Computer Vision API.
    Returns {face_rectangle: celebrity_name} dictionary.
    """
    with open(image_path, 'rb') as img_file:
        img_data = img_file.read()

    url = f"{VISION_ENDPOINT}/vision/v3.2/analyze?visualFeatures=Categories&details=Celebrities"


In [ ]:
def process_photos(confidence_threshold=0.7):
    total_photos = 0
    total_faces = 0
    unique_faces = set()
    detected_face_names = set()

    Path(destination_folder).mkdir(exist_ok=True)
    no_face_dir = Path(destination_folder) / "No Faces Detected"
    no_face_dir.mkdir(exist_ok=True)

    for filename in os.listdir(source_folder):
        if not filename.lower().endswith(('jpg', 'jpeg', 'png', 'bmp')):
            continue

        total_photos += 1
        image_path = os.path.join(source_folder, filename)

        with open(image_path, 'rb') as image_stream:
            faces = face_client.face.detect_with_stream(
                image=image_stream,
                return_face_id=True,
                detection_model='detection_03',
                recognition_model='recognition_04'
            )

        if not faces:
            shutil.copy2(image_path, no_face_dir / filename)
            continue

        total_faces += len(faces)
        face_ids = [face.face_id for face in faces]
        unique_faces.update(face_ids)

        identify_results = face_client.face.identify(face_ids, PERSON_GROUP_ID)
        celeb_map = get_celebrities(image_path)

        faceid_to_name = {}

        for face, identify_result in zip(faces, identify_results):
            name = None

            candidates = identify_result.candidates
            if candidates:
                candidate = candidates[0]
                if candidate.confidence > confidence_threshold:
                    person = face_client.person_group_person.get(PERSON_GROUP_ID, candidate.person_id)
                    name = person.name

            if name is None:
                for rect, celeb_name in celeb_map.items():
                    if approximately_same_face(face.face_rectangle.as_dict(), rect):
                        name = celeb_name
                        break

            if name is None:
                name = face.face_id.replace('-', '')

            faceid_to_name[face.face_id] = name
            detected_face_names.add(name)

        output_dirs = set(faceid_to_name.values())
        for folder_name in output_dirs:
            safe_folder = folder_name.replace('/', '_')
            dest_path = Path(destination_folder) / safe_folder
            dest_path.mkdir(exist_ok=True)
            shutil.copy2(image_path, dest_path / filename)

    print(f"Number of photos processed: {total_photos}")
    print(f"Number of unique faces detected: {len(unique_faces)}")
    print(f"Total faces detected: {total_faces}")
    print(f"Face names / IDs detected: {list(detected_face_names)}")


In [ ]:
process_photos()

Number of photos processed: 0
Number of unique faces detected: 0
Total faces detected: 0
Face names / IDs detected: []
